# Q2. Energy model with Accelergy

In [ ]:
# This code allows you to generate the Energy plots for DRAM, SRAM, register files, and MAC.
# How to run: Afer instaling timeloop-accelergy and connecting to drive, the code should be run in the main.ipynb notebook

# Parameters:
#   component_class : Name of component used (ie. DRAM, SRAM, etc)
#   depth           : Depth of selected component
#   width           : Width of selected component
#   end_width_depth : Value to which you will vary either depth or width to
#   steps           : Steps between the sampled value of the varying parameter.
#   what_vary       : The parameter being varied ("width" or "depth")

# =======Parameters================
component_class="SRAM"
depth=8192
width=16
end_width_depth= 2048
steps = 24
what_vary = "width"
#=========================================
path="/content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/arch_base.yaml"
j = 0
if what_vary == "width":
  reads_write_var = torch.empty(int((end_width_depth-width)/steps)+1, 3) # To store results
  for change_width in range(width, end_width_depth+1, steps):
    YAML_generator(component_class, depth, change_width, path)
    !timeloop-model /content/arch.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/prob.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/map.yaml
    energy_read, energy_write = YAML_parser("/content/timeloop-model.ERT_summary.yaml")
    reads_write_var[j,0] = energy_read
    reads_write_var[j,1] = energy_write
    reads_write_var[j,2] = change_width
    j +=1
elif what_vary == "depth":
  reads_write_var = torch.empty(int((end_width_depth-depth)/steps)+1, 3) # To store results
  for i in range(depth, end_width_depth+1, steps):
    YAML_generator(component_class, depth, width, path)
    !timeloop-model /content/arch.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/prob.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/map.yaml
    energy_read, energy_write = YAML_parser("/content/timeloop-model.ERT_summary.yaml")
    energy_read, energy_write = YAML_parser("/content/timeloop-model.ERT_summary.yaml")
    reads_write_var[j,0] = energy_read
    reads_write_var[j,1] = energy_write
    reads_write_var[j,2] = depth
    j +=1
    depth +=i
else:
  print("Error: No such vary dimention")

# Plot
read = reads_write_var[:, 0].numpy() # Tensor to numpy array
write = reads_write_var[:, 1].numpy()
var = reads_write_var[:, 2].numpy()

plt.figure(figsize=(10, 6))
plt.plot(var, read, label='Reading')
plt.plot(var, write, label='Writing')
plt.xlabel('Width')
plt.ylabel('Energy (J)')
plt.title('Energy vs Width')
plt.legend()
plt.grid(True)
plt.show()



# intermac
The code for intermac required adjustment to collect more data because the timeloop-model.ERT_summary.yaml for intermac is different.

In [ ]:
# How to run: Because the intermac timeloop-model.ERT_summary.yaml file generates 3 energies (energy_read, energy_write , energy_gated),
# it is important to first modify the YAML_parser function to identify these energies. The modified YAML_parser() is shown after this code.

component_class="intmac"
depth=512
width=2
end_width_depth= 64
steps = 1
what_vary = "width"
path="/content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/arch_base.yaml"
j = 0
if what_vary == "width":
  reads_write_var = torch.empty(int((end_width_depth-width)/steps)+1, 4) # To store results
  for change_width in range(width, end_width_depth+1, steps):
    YAML_generator(component_class, depth, change_width, path)
    !timeloop-model /content/arch.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/prob.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/map.yaml
    energy_read, energy_write, energy_gated = YAML_parser("/content/timeloop-model.ERT_summary.yaml")
    reads_write_var[j,0] = energy_read
    reads_write_var[j,1] = energy_write
    reads_write_var[j,2] = change_width
    reads_write_var[j,3] = energy_gated
    j +=1
elif what_vary == "depth":
  reads_write_var = torch.empty(int((end_width_depth-depth)/steps)+1, 3) # To store results
  for change_depth in range(depth, end_width_depth+1, steps):
    YAML_generator(component_class, change_depth, width, path)
    !timeloop-model /content/arch.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/prob.yaml /content/gdrive/MyDrive/PhD_ETS/ELE6310E/Colab_Notebooks/HW/A2/Q2/map.yaml
    energy_read, energy_write, energy_gated = YAML_parser("/content/timeloop-model.ERT_summary.yaml")
    reads_write_var[j,0] = energy_read
    reads_write_var[j,1] = energy_write
    reads_write_var[j,2] = change_depth
    reads_write_var[j,3] = energy_gated
    j +=1
else:
  print("Error: No such vary dimention")

# Plot

read = reads_write_var[:, 0].numpy() # Tensor to numpy array
write = reads_write_var[:, 1].numpy()
var = reads_write_var[:, 2].numpy()
gate = reads_write_var[:, 3].numpy()

plt.figure(figsize=(10, 6))
plt.plot(var, read, label='mac_random')
plt.plot(var, write, label='mac_reused')
plt.plot(var, gate, label='mac_gated')
plt.xlabel('Width')
plt.ylabel('Energy (J)')
plt.title('Energy vs Width')
plt.legend()
plt.grid(True)
plt.show()

# YAML_parser (Modified)

In [ ]:
def YAML_parser(path="timeloop-model.ERT_summary.yaml"):
    current_path = os.getcwd()
    path = os.path.join(current_path, path)
    with open(path, 'r') as stream:
        data_loaded = yaml.safe_load(stream)
    for data in data_loaded['ERT_summary']['table_summary'][1]['actions']:
        if data['name'] == 'read':
            if "max_energy" in data.keys():
                energy_read = data['max_energy']
            else :
                energy_read = data['energy']
        if data['name'] == 'write':
            if "max_energy" in data.keys():
                energy_write = data['max_energy']
            else :
                energy_write = data["energy"]

        if data['name'] == 'mac_random':
            energy_read = data['energy']
        if data['name'] == 'mac_reused':
            energy_write = data['energy']
        if data['name'] == 'mac_gated':
            energy_gated = data['energy']

    return energy_read, energy_write , energy_gated